## Apprivoisez les Pandas et leurs _DataFrames_

### Généralités

Dr. X est un grand habitué des tableurs, comme Origin, Excel et Calc. Le format tableau permet de visualiser efficacement les données, de les manipuler et d'avoir une grande flexibilité sur leur contenu (texte, nombres, dates). Dr. X ne se perd pas dans ses tableaux grâce aux en-têtes des colonnes, et peut les classer, les filtrer selon ses goûts. Dr. X apprécie la possibilités de tracer des graphes à la volée, même si ceux-ci ne sont pas tout à fait de qualité _publication_. Dr. X est également heureux de savoir que ses fichiers sont rapidement exportables en un format texte, de type `.csv` ou  `.tsv`.

Dr. X sera donc ravi d'en apprendre plus sur les `Pandas.Dataframe`, un type d'objet qui vont s'utiliser comme des tableurs en Python. Nous verrons dans ce tutorial : 
* comment [créer, importer et exporter des `DataFrame`](#acceder-aux-donnees-dun-dataframe) à partir de dictionnaires Python ou de fichiers `.csv` et Excel
* comment [accéder aux données d'un `DataFrame`](#acceder-a-un-dataframe) pour afficher une ligne, une colonne ou une sélection arbitraire
* comment [manipuler les données au sein de ces `DataFrame`](#modifier-les-dataframes) pour modifier des valeurs, ajouter ou supprimer des colonnes et des lignes ou bien les trier.
* comment [effectuer des opérations de groupe](#operations-de-groupe) sur les données, notamment les `groupby()`, `agg()` pour exploiter la puissance totale des DataFrames !
* comment [fusionner deux `DataFrame` et plus](#fusionner-deux-dataframe-avec-merge-join-et-concat) avec les méthodes `.merge()`, `.join()`, `.concat()` !

L'objectif va être de 'retrouver les sensations d'Excel' le plus rapidement possible. 

----------------------------------

### Creer un DataFrame

Il est possible de créer un `DataFrame` _ex nihilo_ à partir d'un dictionnaire Python, mais nous verrons rapidement que cela n'est probablement pas la manière la plus efficace de les générer. Nous verrons rapidement ensuite comment les importer, par exemple depuis un fichier `.csv`. 

#### À partir d'un dictionnaire ou d'un tableau NumPy

Reprenons notre inénarrable exemple de notre groupe de TD et de ses notes. Cela confine à l'obsession, mais c'est pour la bonne cause. Créons un dictionnaire contenant quatre clés : 

* `Prénom`
* `Nom`
* `TD_1`, la note obtenue au TD 1
* `TD_2`, la note obtenue au TD 2

Compte tenu de la manière dont on a créé notre dictionnaire, il va être assez difficile, par exemple, d'obtenir la note de `Debasish` au TD2. Il faudrait, pour cela, retrouver l'_indice_ auquel on trouve `Debasish` dans la `list` correspondant à `first_name` et examiner la note de l'indice correspondant dans `TD_2`. 

Il existe une fonction [numpy.where()](https://numpy.org/doc/stable/reference/generated/numpy.where.html) qui renvoie un tel indice à partir d'un objet `np.ndarray`, mais vous pouvez constater que cette solution n'est pas très intuitive.

In [ ]:
import numpy as np

my_dict = {'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[18,14,13,11,16,12],
            'TD_2':[13,17,15,13,10,9],
            }

### Numpy solution
# Find index where `Debasish` is found in `first_names`
# Note : np.where returns two arguments, one being empty if we are looking in 1D arrays
idx_match,  = np.where(np.array(my_dict['first_name']) == 'Debasish') 
grade = my_dict['TD_2'][idx_match[0]] # 
print('Grade is : ' + str(grade))

C'est ici que les `DataFrame` vont nous aider ! On peut les créer directement à partir d'un dictionnaire, d'un tableau `np.ndarray`, ou bien d'autres types de données. La plupart du temps, on peut simplement en créer un en invoquant la commande `pd.DataFrame()` et en mettant en entrée les données que vous voulez voir converties. Regardons ce que ça donne si on essaie avec un dictionnaire : 

In [ ]:
import pandas as pd             # Not the best shorthand for French people ...

my_df = pd.DataFrame(my_dict)
my_df

Vous voyez ici déjà que l'affichage de vos données est plutôt joli par rapport à ce que Python vous sert en général, et ressemble assez furieusement à un tableau Excel. Chouette, non ? Le format a également l'air de bien s'accommoder de mélanger des chaînes de caractère et des chiffres, mais ça, c'était déjà le cas avec les listes et bien d'autres types de variables en Python.

__Exercice__ : Essayez de faire la même chose à partir d'un tableau `np.ndarray` à plusieurs dimensions, et regardez comment les données s'organisent. 

In [ ]:
# Your beautiful code here

#### À partir d'un fichier .csv, voire Excel

La meilleure manière de créer des `pd.DataFrame` reste cependant des les importer depuis un fichier formatté avec la bien-nommée fonction `pd.read_csv()`. Essayons de lire mon fichier de rhéologie préféré, `rheology.csv`, qui se trouve dans le sous-dossier `files` et qui a le mauvais goût d'être formatté à la _française_, c'est à dire avec des `';'` comme séparateurs entre colonnes et des `','` comme séparateurs des décimales. 

In [ ]:
my_rheol = pd.read_csv('files/rheology.csv') # 'What could go wrong ?'
my_rheol                                     # Really wrong things

C'est le bazar complet, mais ne vous inquiétez pas, il est possible de spécifier à la fois : 

* Le délimiteur des champs, `sep` ou `delimiter`
* Le séparateur des décimales, `decimal`
* Un nombre de lignes à ignorer au début du fichier, `skiprows`

Il existe [bien d'autres options](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) si vous avez encore des problèmes pour des types de fichiers ou des manières de les lire bien particulières.

In [ ]:
my_rheol = pd.read_csv('files/rheology.csv', decimal=',', sep=';')
my_rheol

Le résultat est bien plus satisfaisant ! 

Si vous ne voulez _vraiment_ pas vous embêter avec ces paramètres régionaux, vous pouvez également simplement importer un fichier _Excel_ avec `pd.read_excel()`. Celui-ci 'comprend' les fichiers : 
* `.xls`, par l'intermédiaire du module `xlrd`
* `.xlsx`, par l'intermédiaire du module `openpyxl`
* `.ods`, par l'intermédiaire du module `odfpy` 

Dans notre espace de travail, seul `openpyxl` est installé, il ne vous sera donc pas possible d'importer d'autres types de fichiers que `.xslx`. 

In [ ]:
my_data = pd.read_excel('files/rheology.xlsx')
my_data

#### Exporter un DataFrame

D'une manière pas tout à fait symétrique aux fonctions `pd.read_csv()` et `pd.read_excel()`, il va être possible d'exporter un DataFrame dans ces formats grâce aux _méthodes_ `.to_csv()` et `.to_excel()`  :  

In [ ]:
my_data.to_csv('./files/my_new_csv_file.csv')
my_data.to_excel('./files/my_new_excel_file.xlsx')

---------------------

### Acceder a un DataFrame

Le format tableur est bien joli, mais il serait quand même chouette de pouvoir récupérer les notes de notre élève de manière efficace, non ? Voire de modifier les valeurs du tableau en cas d'erreur. 

#### Accéder au début ou à la fin d'un `DataFrame` avec `.head()` et `.tail()`

Parfois, lorsque votre `DataFrame` est un peu long, il serait utile d'essayer d'effectuer le traitement des données que vous avez laborieusement développé seulement sur les $n$ premières lignes de votre `DataFrame`, ou sur les $n$ dernières. Pour cela, vous pouvez employer les méthodes `.head()` et `.tail()`. Notez qu'elles renvoient bien des sous-parties du `DataFrame` initial, ce n'est pas juste une 'option d'affichage' !

In [ ]:
import pandas as pd
my_rheol = pd.read_csv('files/rheology.csv', decimal=',', sep=';')

beginning = my_rheol.head(10)
beginning

#### Indexation standard (`[]`)

Pour accéder à une colonne de notre tableau, on peut travailler comme avec les dictionnaires, ou comme avec des objets. Accédons aux prénoms de notre `pd.DataFrame` associé à notre inénarrable classe de TD :

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[18,14,13,11,16,12],
            'TD_2':[13,17,15,13,10,9]})

my_df['first_name']   # Does work 
# my_df.first_name        # Would also have worked fine, unless column has a weird name, e.g. '1' 

Ce que renvoie notre commande `my_df['first_name']` ressemble à un tableau Numpy `np.ndarray` ou une liste, mais ce n'est pas tout à fait le cas, car à gauche vous voyez une liste d'indices. C'est parce que cet objet est en fait une `Series` Pandas, la version '1D' des `DataFrame`. On peut d'ailleurs les convertir très simplement en tableaux Numpy, par exemple ici en écrivant `np.array(my_df['first_name'])`. 

On peut assez facilement sélectionner deux ou plusieurs colonnes parmi toute notre série de données. Pour cela, on remplace notre nom de colonne (par exemple, `'first_name'`) par une liste (par ex., `['first_name', 'last_name']`) et le tour est joué ! 

In [ ]:
my_names = my_df[['first_name', 'last_name']]   # Don't forget the extra square brackets !
print(my_names)

Comme pour les `np.ndarray` et les listes, on peut effectuer des coupes au niveau des colonnes avec la syntaxe `[a:b:step]`, comme par exemple ici si je ne veux regarder qu'un étudiant sur deux (même si cela est assez curieux comme pratique). Cela est possible tant que les indices du `DataFrame` sont bien ordonnés de $0$ à $n$ et qu'on n'a pas donné _explicitement_ de nom à cette colonne des indices : 

In [ ]:
my_df[::2]

La puissance des `pd.DataFrame` vient du fait que l'on peut __cumuler  facilement les sélections sur les lignes et les colonnes__. On peut tout à fait effectuer la chose suivante : 

In [ ]:
my_df['first_name'][::2]

#### Indexation booléenne 

Enfin, comme pour les [`np.ndarray`](./Application_A_Numpy.ipynb#Indexation-logique), on peut construire des _conditions_ sur construire des sur les valeurs prises dans les cases de notre tableau et effectuer des _coupes_ à partir de ces conditions. _Hein ?_ Voyez plutôt, si je veux récupérer la note de Serguey au TD1. Je vais commencer par trouver la _ligne_ correspondant à Serguey dans la liste des prénoms, puis, à partir de cette sélection de mon jeu de données initiale, faire une _nouvelle coupe_ permettant de retrouver les notes de TD1. Fort pratique ! 

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]})
            
is_serguey = my_df['first_name'] == 'Serguey'
my_grade = my_df[is_serguey]['TD_1']
my_grade

Je peux même demander à appliquer _plusieurs_ conditions booléennes. Essayons par exemple de trouver un.e éventuel.le élève qui a moins que la moyenne aux deux TD, un élève en _difficulté_, même si ici les résultats sont plutôt bons. On utilisera alors [les opérateurs `~`, `&`, `|` et `^`](./Application_A_Numpy.ipynb#Combinaisons-de-conditions-sous-Numpy), et des fonctions Numpy, qui s'appliquent tout à fait correctement sur les colonnes de mon `pd.Dataframe`. 

In [ ]:
belowavg_td1 = my_df['TD_1'] < np.mean(my_df['TD_1'])       # Values below average of all values for TD1
belowavg_td2 = my_df['TD_2'] < np.mean(my_df['TD_2'])       # Values below average of all values for TD2
my_df[belowavg_td1 & belowavg_td2]

__Exercice__ :

* Pour le fichier de rhéologie `'files/rheology.csv'`, essayez de tracer avec [Matplotlib](./Application_B_Matplotlib.ipynb) la courbe contrainte ('Shearstress') vs. taux de déformation ('Shearrate') correspondant à l'étape 'Flow Curve Down Forward' (contenue dans la colonne 'Name').

In [ ]:
# Pandas and Matplotlib ... Fuuuuuuussssiiiioooonnnn ! 

#### Encore plus de flexibilité : l'indexation avec `.loc[]`

Pandas recommande d'utiliser cette fonction afin d'indexer les `pd.DataFrame`. Hormis le fait que cette 'fonction' soit suivie de crochets `[` et `]` et non de parenthèses `(` et `)`, celle-ci fonctionne d'une manière très similaire à l'indexation simple et booléenne. La syntaxe est en fait encore un peu plus flexible  et permet de combiner plusieurs conditions (par exemple, un nom de colonne et une sélection sur les lignes) entre les mêmes crochets, ce qui n'est pas possible avec l'indexation simple `[]`.


In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]})

is_serguey = my_df['first_name'] == 'Serguey'
my_grade_loc = my_df.loc[is_serguey, 'TD_1']    # Combining two conditions in bet
# my_grade_old = my_df[is_serguey, 'TD_1']        # This does not work

print(my_grade_loc)

Nous verrons plus loin que les indexations avec `[]` et avec `.loc[]` fonctionnent _différemment lorsqu'on essaie de modifier des valeurs_ du `pd.DataFrame`.

_Note_ : Vous pouvez très bien sélectionner plusieurs colonnes. Dans ce cas, il faut faire une `list` des colonnes à afficher, que ce soit pour la méthode en `.loc[]` ou pour l'indexation directe : 

In [ ]:
is_serguey = my_df['first_name'] == 'Serguey'
columns = ['TD_1', 'TD_2']

print('.loc[] result : ')
serguey_grades_loc = my_df.loc[is_serguey, columns]
print(serguey_grades_loc)

print('Direct indexing result :')
serguey_grades_dir = my_df[is_serguey][columns]
print(serguey_grades_dir)

#### Selectionner une ligne ou plusieurs lignes a la main avec `.iloc[]`

Parfois, il vous sera nécessaire de sélectionner _à la main_ une ligne de votre tableau indépendamment de toute condition. Par défaut, les lignes des `DataFrames` sont indexées par des entiers dans une colonne 'sans nom', ce qui permet d'écrire :

```
    my_df[0]
```
pour demander à accéder à la première ligne. 

Mais parfois, vos DataFrames auront des indices qui ne sont pas dans l'ordre, des indices explicitement nommés _voire_ des indices qui ne sont pas des chiffres ! Voyons un exemple pour des lignes qui ne sont pas _indexées dans l'ordre_, et essayons naïvement de demander la première ligne du tableau à l'aide de l'indexation normale `[]` puis avec `.loc[]` : 

In [ ]:
import pandas as pd

my_sneaky_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]}, index=[5,6,3,2,1,4])

# my_sneaky_df[0]          # Fails miserably
# my_sneaky_df.loc[0]        # Does not work either

Aïe ! L'indexation normale `[]` renvoie carrément une erreur, tandis que la version avec `.loc[]` renvoie naturellement la ligne correspondant à l'indice 0, mais qui n'est plus la première ligne de mon tableau. 

C'est pour ce genre de cas que la fonction `.iloc[]` existe. On peut lui préciser un indice seul, mais également des _coupes_ du type `[start:stop:step]` et des listes d'indices. Commentez et dé-commentez les lignes suivantes pour examiner le fonctionnement de `.iloc[]`. 

In [ ]:
my_sneaky_df.iloc[0]
# my_sneaky_df.iloc[::3] # Also works 
# my_sneaky_df.iloc[[0,2,4]] # Works too ! Yay !

------------------------------

### Modifier les DataFrames

Dans cette section, nous allons maintenant voir comment _modifier_ des DataFrames, c'est à dire comment : 

* modifier des valeurs individuelles
* rajouter ou supprimer des colonnes ou des lignes
* assembler des DataFrames entre eux, notamment avec `pd.concat()`

Pour bien comprendre comment fonctionnent certaines de ces opérations, je vous conseille de lire d'abord la section sur la [mutabilité](./Tutorial_2_ListsTuplesDicts.ipynb#Python-et-la-mutabilite) en Python. Mais ... normalement vous l'avez déjà fait :-) .

#### Modifier des valeurs individuelles dans un DataFrame 

Reprenons notre exemple de classe de TD. Je me suis trompé sur la note d'Aika au TD2, et je voudrais la corriger rapidement. Rien de plus simple, non ? J'accède à la bonne ligne en cherchant `my_df['first_name'] == 'Aika'` et la bonne colonne en choisissant `TD2` et le tour est joué, non ?

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]})

aika_row = my_df['first_name'] == 'Aika'
my_df[aika_row]['TD_1'] = 16
my_df

Horreur ! Cela ne fonctionne pas, notre `DataFrame` initial reste inchangé. Vous remarquereez que Pandas est assez sympathique tout de même, car il vous indique la marche à suivre pour que votre code fonctionne : __pour changer des valeurs d'un `DataFrame` indexé à la fois sur les lignes et les colonnes, il faut utiliser `.loc[]`__. La raison derrière tout ça est que bien souvent, Pandas effectue une _copie_ quand on demande à voir `my_df[my_row]`. On modifie alors la copie, et non l'objet original, d'où le fait qu'on ait l'impression que l'opération ne fonctionne pas. Essayons avec `.loc[]` :

In [ ]:
my_df.loc[aika_row,  'TD_1'] = 16
my_df

On peut, bien entendu, changer de multiples valeurs d'un coup, par exemple plusieurs colonnes ou plusieurs lignes, si par exemple on avait _vraiment_ rempli au radar les notes de TD des élèves :-) 

In [ ]:
tds_columns = ['TD_1', 'TD_2']
my_students = (my_df['first_name'] == 'Olivier') | (my_df['first_name'] == 'Lu') # Parentheses are important here, otherwise operator "|" applies first

my_df.loc[aika_row, tds_columns] = [15,13]      
my_df.loc[my_students, 'TD_1'] = [20,20]        # Just needs to be of a "compatible" shape
print(my_df)

Les notes d'Olivier, d'Aika et de Lu ont bien été modifiées dans le `DataFrame` original.

#### Ajouter des lignes et des colonnes 

Cette fois-ci, rassurez-vous, il n'y aura pas de pièges. Si vous voulez rajouter une nouvelle colonne `TD_3` à votre `DataFrame`, il suffit de la préciser. Vous pouvez choisir de spécifier toutes les valeurs, ou juste une seule pour tout le monde :

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]})

my_df['TD_3'] = [11,14,13,18,18,16]
my_df['TD_4'] = 18                      # A very lazy grading scheme indeed 
my_df

#### Supprimer des lignes ou des colonnes 

Si vous voulez faire des coupes sombres dans données, voire des coupes claires pour y ... voir plus clair, vous pouvez cette fois-ci utiliser la méthode `.drop()` qui est bien implémentée en Pandas. Essayons d'oublier les notes du TD 2, puis ensuite Serguey Prigogine car il éclipse un peu trop le reste de la classe avec ses trop bons résultats. Il faudra une fois de plus rajouter l'option `inplace=True` pour directement modifier le `DataFrame` original :

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9],
            'TD_3': [11,12,10,11,13,12],
            'TD_4': 14})


my_df.drop('TD_2', axis='columns', inplace=True)    # We remove a column : specify axis='columns'
my_df.drop(1, axis='index', inplace=True)           # We remove a row : specify axis='index'
my_df

Si votre `DataFrame` a été ré-indicé, par exemple avec 'ID', il faut quand même spécifier qu'on supprimer des lignes avec l'option `axis='index'`, mais il faut préciser les indices _réels_. Voyez ici l'exemple pour des étudiants numérotés de 22 à 27 : 

In [ ]:
my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9],
            'ID':[22,23,24,25,26,27]})
my_df.set_index('ID', inplace=True)
my_df.drop([22,24], axis='index', inplace=True)
my_df

__Exercice__ : 

* Essayez de calculer la moyenne de chaque élève et rajoutez-la à la liste de leurs notes. Vous pouvez pour cela utiliser la méthode `.mean()` disponible pour les DataFrames. Essayez également de calculer la moyenne du groupe de TD pour chaque TD. 
* Les notes du TD 1 et du TD 2 ont été inversées. Trouvez un moyen de remédier au problème avec la méthode `.drop()`. Lisez ensuite la page d'aide associée à la méthode `.rename()` et essayez de trouver une solution plus _élégante_ au problème.

#### Trier des données dans un `pd.DataFrame` 

Les fonctions de tri des `DataFrame` sont assez puissantes et vous permettent de trier rapidement vos données selon un critère ou même une liste de critères (par ordre de priorité). 
* Pour trier simplement sur les indices (les 'lignes'), vous pouvez appeler `.sort_index()` avec pour argument `axis=index` (`axis=columns` vous permet de trier les colonnes, si cela vous intéresse ...). 
* Pour trier selon une ou plusieurs valeurs, on peut utiliser `.sort_values()`, qui prend un argument `by`. On peut lui passer un nom de colonne, par exemple, `last_name` pour trier par ordre alphabétique, ou une liste de noms de colonnes si on veut trier selon de multiples critères, par exemple `['last_name', 'first_name']` pour d'abord trier sur les noms de famille, puis ensuite sur les prénoms en cas d'égalité. Ces fonctions peuvent sans problème trier des chaînes de caractère et des nombres entre eux, mais va bien entendu peiner à dire si `'toto'` se situe avant 12 et après 13.

Dans les deux cas, si vous voulez modifier le `DataFrame` (et non pas créer un nouvel objet), vous devrez spécifier l'option `inplace=True` lors de l'appel à la méthode. Si vous voulez trier les valeurs dans l'ordre décroissant, vous pouvez également rajouter l'option `ascending=False`.

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu', 'Françine' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai', 'Margoulin'],           # Françine and Olivier are unrelated, so they say ...
            'TD_1':[11,14,13,15,12,15,14],
            'TD_2':[13,17,15,14,16,9,12],
            'TD_3': [11,12,10,12,13,12,18]})

print('Sort by last name only')
print(my_df.sort_values(by='last_name'))             

print('Sort by last then first name')
print(my_df.sort_values(by=['last_name', 'TD_3']))   # Last   

__Exercice__ : 

* Triez les élèves par notes de TD, en mettant la priorité sur le TD 1 puis le TD 3 et enfin le TD 2.
* Essayez de créer _automatiquement_ le rang de chaque élève (son classement général) à partir de ses notes de TD. Puis utilisez ce rang pour indexer votre `DataFrame`. __Esprit prépa__ !!!!!

----------------------

### Operations de groupe

Les `DataFrames` Pandas expriment toute leur puissance avec les opérations de groupe. Mais ... c'est quoi ?

#### La méthode  `groupby()`

Imaginons que notre professeur (toujours aux intentions douteuses) officie dans deux classes distinctes, la Terminale A et la Terminale B (abrégées en TA et TB). Il ou elle souhaite obtenir des statistiques sur ses classes, mais n'a pas envie de s'embêter à mettre ses données dans des fichiers séparés.

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu', 'Francine', 'Jérémie', 'Marco', 'Jorrit', 'Oyeniyi', 'Elisabeth', 'Apichatpong'],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai', 'Margoulin', 'Fassolle', 'Leonarduzzi', 'Peeters', 'Oyewunmi', 'Battenberg', 'Weerasethakul'],           # Françine and Olivier are unrelated, so they say ...
            'TD_1':[11,14,13,15,12,15,14,15,9,11,13,11,12],
            'TD_2':[13,17,15,14,16,9,12,9,11,13,9,12,11],
            'TD_3': [11,12,10,12,13,12,18,11,12,10,11,13,12],
            'class': ['TA', 'TA', 'TB', 'TA', 'TB', 'TB', 'TA', 'TB', 'TA', 'TB', 'TB', 'TA', 'TA']})

Comment effectuer une moyenne (et éventuellement une médiane, un écart-type, etc.) sans s'embêter ? Le `groupby()` va venir nous aider !

In [ ]:
grouper = my_df.groupby(by='class')
grouper

> Euh ... oui, merci, cela ne m'aide pas trop en fait. 

Attendez un petit moment ! On va pouvoir appliquer des méthodes à notre ami `grouper`:

- `.mean()`, la moyenne
- `.std()`, la déviation standard
- `.first()`, le premier élément 
- `.last()`, le dernier élément
- `.cumsum()`, la somme cumulée
- `.median()`, la médiane
- `.min()`, `.max()`, ...

On obtient alors un `pd.Series()` qui a pour index les deux classes `TA` et `TB`: 

In [ ]:
grouper['TD_1'].mean()

Notez qu'on peut effectuer des opérations sur plusieurs colonnes, et on se retrouve alors avec un `pd.DataFrame()`, une fois de plus avec les indices `TA` et `TB` :

In [ ]:
grouper[['TD_1', 'TD_2']].mean()

On peut également effectuer le groupage sur toutes les colonnes, mais dans ce cas __attention__, il faut que la méthode d'agrégation, c'est à dire le `.mean()`, le `.first()`, __soit compatible avec toutes les colonnes du `DataFrame` ! 

In [ ]:
grouper.first() # Works
# grouper.mean() # Breaks down

#### Agrégateurs différents par colonne : la méthode `.agg()`

Encore plus puissant, on peut également décider de faire des statistiques différentes sur les différentes colonnes lorsqu'on agrège. On peut même demander _deux agrégations différentes pour une même colonne_. Je vous montre ci-dessous comment faire pour renvoyer le prénom du premier élève de la classe, la moyenne du TD1 et la déviation standard du TD1 et du TD2. On commence à observer la puissance des `DataFrame` ! Dans ce cas, vous pouvez passer un dictionnaire à la méthode `.agg()`, ou une liste de `kwargs` (arguments à clé) avec une syntaxe un peu bizarre que je détaille ci-dessous.

In [ ]:
aggregated = grouper.agg(first_student=('first_name', 'first'), # New column name = (old column name, operation)
            TD_1_avg=('TD_1', 'mean'),
            TD_1_std=('TD_1', 'std'),
            TD_2_std=('TD_2', 'std')) 

aggregated

Vous avez noté que cette fois-ci, le nom des opérations est une chaîne de caractères. Si la fonction existe directement dans Python (par exemple, `sum`), vous pouvez également directement indiquer celle-ci à la place d'une chaîne de caractères. Voire ... vous pouvez lui donner une fonction que vous auriez codée, ou une fonction `lambda` !

Vous pouvez ensuite ré-insérer les valeurs de votre moyenne dans le `DataFrame` original ! Pour cela, vous devrez faire en sorte que votre `DataFrame` original et celui agrégé aient les mêmes indices pour les lignes. Vous pouvez alors utiliser la méthode `set.index()` du `DataFrame` original pour arriver à vos fins. Voyons comment cela se présente : 

In [ ]:
my_df = my_df.set_index('class')
my_df['TD_1_avg'] = aggregated['TD_1_avg']
my_df

**Exercices**

La colonne `Shearstrain` du fichier de rhéologie suivant ne se réinitialise pas à chaque changement de palier (`step`). La colonne `Name` indique le nom de chaque palier (`step`) et est unique à chaque palier. 

1. Tentez de créer une colonne `step` correspondant au numéro de palier dans la séquence. (__Indice__ : si vous ne voulez pas trier vos colonnes lorsque vous effectuez un `.groupby()`, rajoutez-lui l'argument supplémentaire `sort=False`)

2. Essayez de modifier `Shearstrain` pour que celle-ci soit réinitialisée à 0 à chaque palier, ou créez une nouvelle colonne.

In [ ]:
my_rheol = pd.read_csv('files/rheology.csv', decimal=',', sep=';')

# You can doooooooooo iiiiiiiiittttttttt mmmmmmyyyyyyy frrrrriiiieeeennndddd 

### Fusionner deux DataFrame avec merge, join et concat

#### Merge : jointure via nom de colonne 

Imaginons maintenant que notre cher collègue, professeur de Tagalog, est également fanatique de Pandas. Chacun d'entre vous a les mêmes notations, mais pas forcément les mêmes élèves (bien que vous ayez des élèves en commun) ... et vous avez quand même deux `DataFrame` différents correspondant aux notes des élèves dans les deux matières (Physique et Tagalog). Les étudiants sont identifiés par un `id` unique : 

In [120]:
import pandas as pd

df_phys = pd.DataFrame({'id': [720,365,256,130,899,477],
            'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu'],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],           # Françine and Olivier are unrelated, so they say ...
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9],
            'TD_3': [11,12,10,12,13,12],
            'class': ['TA', 'TA', 'TB', 'TA', 'TB', 'TB']})

df_tglo = pd.DataFrame({'id': [130,365,720,131,988,753],
                        'first_name':['Debasish', 'Serguey', 'Olivier', 'Matilda', 'Harry', 'Inigo'],
                        'last_name':['Sarker', 'Prigogine', 'Margoulin', 'May', 'Potter', 'Montoya'],
                        'Partiel_1':[14,10,9,16,8,11],
                        'Partiel_2':[9,11,18,12,10,5],
                        'class':['TA', 'TA', 'TA', 'TC', 'TC', 'TC']})

df_phys.head()

,id,first_name,last_name,TD_1,TD_2,TD_3,class
0,720,Olivier,Margoulin,11,13,11,TA
1,365,Serguey,Prigogine,14,17,12,TA
2,256,Aika,Tokogawa,13,15,10,TB
3,130,Debasish,Sarker,15,14,12,TA
4,899,Muhammad,Ben Amar,12,16,13,TB


In [121]:
df_tglo.head()

,id,first_name,last_name,Partiel_1,Partiel_2,class
0,130,Debasish,Sarker,14,9,TA
1,365,Serguey,Prigogine,10,11,TA
2,720,Olivier,Margoulin,9,18,TA
3,131,Matilda,May,16,12,TC
4,988,Harry,Potter,8,10,TC


On souhaiterait obtenir la moyenne _générale_ des étudiants à partir des notes de ces deux matières. Comment faire ? Eh bien, la méthode `.merge()` des `DataFrame` peut s'en charger ! Cette fonction se comporte un peu comme les [`JOIN` en SQL](https://en.wikipedia.org/wiki/Join_(SQL)) et vont effecuter une _jointure_, qu'on va pouvoir ici effectuer à partir des identifiants uniques. Voyons un peu le résultat : 

In [122]:
df_phys.merge(df_tglo, on='id')     # df_1.merge(df_2, on=<column on which to join>)

,id,first_name_x,last_name_x,TD_1,TD_2,TD_3,class_x,first_name_y,last_name_y,Partiel_1,Partiel_2,class_y
0,720,Olivier,Margoulin,11,13,11,TA,Olivier,Margoulin,9,18,TA
1,365,Serguey,Prigogine,14,17,12,TA,Serguey,Prigogine,10,11,TA
2,130,Debasish,Sarker,15,14,12,TA,Debasish,Sarker,14,9,TA


Bon ! On voit qu'il y a des choses positives, notamment on observe que sur une même ligne, on a désormais les notes des deux matières pour certains étudiants. C'est ce qu'on souhaitait voir. On observe d'autres choses, moins bonnes : 

1. **Certains élèves n'apparaissent pas du tout !** C'est lié en fait à la méthode de jointure qu'on a utilisée. Par défaut, on ne va chercher que les `id` présents dans les __deux__ tableaux. On peut changer ce fonctionnement en modifiant l'argument `how` (par défaut `inner`) : `left` va conserver les clés du tableau de 'gauche' (celui dont on appelle la méthode `.merge()`, et  `outer` garde toutes les clés.

2. **Les noms des élèves et leur classe apparaissent deux fois, dans `first_name_x`, ... et dans `first_name_y` ... ! Ces `_x` et `_y` sont un peu disgracieux**, mais on peut rapidement s'en débarrasser de deux manières : 
   * On peut décider de faire un `merge` sur plusieurs colonnes : `on=['id', 'first_name', 'last_name']`
   * On peut décider d'utiliser l'argument `suffix=(None, '_to_remove')` pour que le tableau de gauche garde ses noms originaux et que l'autre ait un nom explicite qu'on peut retirer ensuite.
  
Voyons donc ce que cela donne : 

In [ ]:
# Left merge 
df_phys.merge(df_tglo, on='id', how='left')

In [ ]:
# Merge on multiple keys
df_phys.merge(df_tglo, on=['id', 'first_name', 'last_name', 'class'], how='outer')

In [ ]:
# Merge on keys with custom suffixes then delete 
merged = df_phys.merge(df_tglo, how='right', on='id', suffixes=(None, '_toremove'))
to_remove = [col for col in merged.columns if 'toremove' in col]
merged.drop(columns=to_remove)

__Remarque__ : les noms de colonne n'ont pas besoin d'être identiques entre le tableau de gauche et celui de droite ! 
Dans ce cas, vous devrez simplement spécifier un `left_on=` et un `right_on=` distincts, et le tour est joué.

#### Join : jointure via les indices

La méthode `.join()`, elle, va effectuer _par défaut_ une jointure _via_ l'`Index` des deux `DataFrame`. Dans notre exemple, on va devoir donc effectuer un `.set_index()` avant d'effectuer le `.join()`, mais dans la vraie vie vous aurez peut-être des `DataFrame` avec les bons indices déjà mis en place !

In [124]:
ind_phys = df_phys.set_index('id')
ind_tglo = df_tglo.set_index('id')

ind_phys.join(ind_tglo, rsuffix='_toto')

,first_name,last_name,TD_1,TD_2,TD_3,class,first_name_toto,last_name_toto,Partiel_1,Partiel_2,class_toto
id,,,,,,,,,,,
720,Olivier,Margoulin,11,13,11,TA,Olivier,Margoulin,9.0,18.0,TA
365,Serguey,Prigogine,14,17,12,TA,Serguey,Prigogine,10.0,11.0,TA
256,Aika,Tokogawa,13,15,10,TB,NaN,NaN,NaN,NaN,NaN
130,Debasish,Sarker,15,14,12,TA,Debasish,Sarker,14.0,9.0,TA
899,Muhammad,Ben Amar,12,16,13,TB,NaN,NaN,NaN,NaN,NaN
477,Lu,Ai,15,9,12,TB,NaN,NaN,NaN,NaN,NaN


#### Concat : concaténer les `DataFrame` 

Dernier cas de figure : j'ai des données de 2023 et de 2024 qui ont les mêmes noms de colonne, et j'aimerais compiler ces données et obtenir un grand `DataFrame` pour toutes les années (éventuellement pour regarder l'évolution des notes moyennes au fil des années ...). Comment faire ? Eh bien, avec un bon petit `pd.concat()` des familles. Notez que cette fois-ci, c'est une fonction de `Pandas`, pas une méthode d'un `DataFrame`. On doit donc donner une `list` (ou un `tuple`) de `DataFrame` à manger à `pd.concat()`, et vous n'êtes pas du tout limités à deux :

In [130]:
df_2023 = pd.DataFrame({'id': [720,365,256,130,899,477],
            'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu'],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],           # Françine and Olivier are unrelated, so they say ...
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9],
            'TD_3': [11,12,10,12,13,12],
            'class': ['TA', 'TA', 'TB', 'TA', 'TB', 'TB'],
            'year':2023})

df_2024 = pd.DataFrame({'id': [723,665,456,830,599,977],
            'first_name':['Jean', 'Cixin', 'Marcos', 'Joko', 'Benazir', 'Jafar'],
            'last_name': ['Moulin', 'Liu', 'Valle', 'Widodo', 'Bhutto', 'Panahi'],          
            'TD_1':[11,12,15,13,12,14],
            'TD_2':[13,14,13,11,17,9],
            'TD_3': [11,13,6,19,11,7],
            'class': ['TD', 'TC', 'TC', 'TD', 'TD', 'TC'],
            'year':2024})

concatenated = pd.concat((df_2023, df_2024))
concatenated 

,id,first_name,last_name,TD_1,TD_2,TD_3,class,year
0,720,Olivier,Margoulin,11,13,11,TA,2023
1,365,Serguey,Prigogine,14,17,12,TA,2023
2,256,Aika,Tokogawa,13,15,10,TB,2023
3,130,Debasish,Sarker,15,14,12,TA,2023
4,899,Muhammad,Ben Amar,12,16,13,TB,2023
5,477,Lu,Ai,15,9,12,TB,2023
0,723,Jean,Moulin,11,13,11,TD,2024
1,665,Cixin,Liu,12,14,13,TC,2024
2,456,Marcos,Valle,15,13,6,TC,2024
3,830,Joko,Widodo,13,11,19,TD,2024


Vous noterez que cette fois-ci, les indices de votre nouveau DataFrame ne sont pas bien intéressants. Si vous le souhaitez, vous pouvez les _réinitialiser_ avec la méthode `.reset_index()`. Si vous voulez complètement vous débarrasser de ces indices et ne pas les transférer dans une nouvelle colonne, précisez en plus `drop=True` : 

In [132]:
concatenated.reset_index(drop=True)

,id,first_name,last_name,TD_1,TD_2,TD_3,class,year
0,720,Olivier,Margoulin,11,13,11,TA,2023
1,365,Serguey,Prigogine,14,17,12,TA,2023
2,256,Aika,Tokogawa,13,15,10,TB,2023
3,130,Debasish,Sarker,15,14,12,TA,2023
4,899,Muhammad,Ben Amar,12,16,13,TB,2023
5,477,Lu,Ai,15,9,12,TB,2023
6,723,Jean,Moulin,11,13,11,TD,2024
7,665,Cixin,Liu,12,14,13,TC,2024
8,456,Marcos,Valle,15,13,6,TC,2024
9,830,Joko,Widodo,13,11,19,TD,2024


Alternativement, vous pouvez décider d'indicer le nouveau tableau avec les `id` des élèves. On peut une fois de plus utiliser `.set_index()` pour l'effectuer, et si vous souhaitez trier les `id` par ordre croissant, vous pouvez appeler la méthode `.sort_index()` qui va ... comme son nom l'indique ... trier les indices :

In [136]:
concatenated.set_index('id').sort_index()

,first_name,last_name,TD_1,TD_2,TD_3,class,year
id,,,,,,,
130,Debasish,Sarker,15,14,12,TA,2023
256,Aika,Tokogawa,13,15,10,TB,2023
365,Serguey,Prigogine,14,17,12,TA,2023
456,Marcos,Valle,15,13,6,TC,2024
477,Lu,Ai,15,9,12,TB,2023
599,Benazir,Bhutto,12,17,11,TD,2024
665,Cixin,Liu,12,14,13,TC,2024
720,Olivier,Margoulin,11,13,11,TA,2023
723,Jean,Moulin,11,13,11,TD,2024


**Exercice 1** : Vous l'avez vu venir ... essayez de déterminer la moyenne générale des élèves pour 2017 et 2018 ! 

In [128]:
# Your code here !

**Exercice 2** : La fonction suivante renvoie un dictionnaire à chaque appel. Appelez cette fonction trois cent fois, environ cent fois par seconde, puis concaténez vos résultats.
Essayez ensuite d'effectuer une moyenne de vos résultats pour chaque seconde, ainsi que l'écart-type `std()` de la distribution de ces valeurs.

In [129]:
from datetime import datetime
from numpy import random
from time import sleep

def fetch():
    now = datetime.now().strftime('%H-%M-%S')
    result = random.choice(50, size=2)
    proba = random.uniform(0,1, size=2)
    return pd.DataFrame({'time':now, 'result':result, 'proba':proba})

########################################################################
### DO NOT TOUCH THE CODE ABOVE THIS LINE ##############################
########################################################################



**Vous voici un.e champion.ne de Pandas !** Voici un emoji pour vous récompenser ! 🐼